# Building a ReAct Agent with GPT and LangChain

In this tutorial, we'll build a **ReAct Agent using GPT models and LangChain**. This provides a production-ready implementation with modern tooling.

## What You'll Learn

- How to use LangChain's agent framework
- How to integrate GPT models with LangChain agents
- How to create and use custom tools
- How LangChain simplifies agent development

## Requirements

This notebook uses:
- **OpenAI GPT models** (gpt-4.1-mini, gpt-4.1-nano, or gemini-2.5-flash)
- **LangChain** for agent orchestration
- **Wikipedia** for information retrieval

---
## 📦 Part 1: Installation

First, let's install the required packages.

In [ ]:
# Install required packages
!pip install -q langchain langchain-openai langchain-community wikipedia python-dotenv

  Preparing metadata (setup.py) ... done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


---
## 🔑 Part 2: Setup OpenAI API

Configure the OpenAI API key. If you're running in an environment with pre-configured API keys, this will use them automatically.

In [ ]:
import os
from random import randint
from dotenv import load_dotenv

# Load environment variables from a .env file
env_path = "/content/drive/MyDrive/.env"
load_dotenv(env_path)

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

---
## 🤖 Part 3: Initialize the LLM

Let's initialize the GPT model using LangChain's ChatOpenAI wrapper.

In [ ]:
from langchain_openai import ChatOpenAI

# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    temperature=0,  # Set to 0 for more deterministic outputs
)

# Test the model
response = llm.invoke("What is the capital of France?")
print(f"Test Question: What is the capital of France?")
print(f"Answer: {response.content}")
print("\n✅ LLM initialized successfully!")

Test Question: What is the capital of France?
Answer: The capital of France is Paris.

✅ LLM initialized successfully!


---
## 🛠️ Part 4: Create Custom Tools

LangChain allows us to create custom tools that the agent can use. Let's create a Wikipedia search tool and a calculator tool.

In [ ]:
from langchain.tools import Tool
from langchain_community.utilities import WikipediaAPIWrapper
import re

# Initialize Wikipedia wrapper
wikipedia = WikipediaAPIWrapper()

# Create Wikipedia tool
def wikipedia_search(query: str) -> str:
    """
    Search Wikipedia and return a summary.
    """
    try:
        result = wikipedia.run(query)
        return result
    except Exception as e:
        return f"Error searching Wikipedia: {str(e)}"

# Create Calculator tool
def calculator(expression: str) -> str:
    """
    Evaluate a Python mathematical expression.
    """
    try:
        # Clean the expression
        expression = expression.strip()
        # Use eval with a restricted namespace for safety
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error evaluating expression: {str(e)}"

print("✅ Custom tools created!")

✅ Custom tools created!


---
## 🔧 Part 5: Define Tools for LangChain

Now let's wrap our functions as LangChain Tools with proper descriptions.

In [ ]:
from langchain.tools import Tool

# Define tools
tools = [
    Tool(
        name="Wikipedia",
        func=wikipedia_search,
        description="Useful for finding factual information about people, places, events, and general knowledge. Input should be a search query."
    ),
    Tool(
        name="Calculator",
        func=calculator,
        description="Useful for mathematical calculations. Input should be a valid Python mathematical expression (e.g., '2 + 2', '68000000 ** 2')."
    )
]

print("Available Tools:")
for tool in tools:
    print(f"\n  📌 {tool.name}")
    print(f"     {tool.description}")

Available Tools:

  📌 Wikipedia
     Useful for finding factual information about people, places, events, and general knowledge. Input should be a search query.

  📌 Calculator
     Useful for mathematical calculations. Input should be a valid Python mathematical expression (e.g., '2 + 2', '68000000 ** 2').


---
## 🧪 Part 6: Test the Tools

Let's test our tools individually before integrating them with the agent.

In [ ]:
# Test Wikipedia
print("Testing Wikipedia Tool:")
print("="*50)
result = wikipedia_search("France population")
print(result[:300] + "...")
print()

Testing Wikipedia Tool:
Page: Demographics of France
Summary: The demography of France is monitored by the Institut national d'études démographiques (INED) and the Institut national de la statistique et des études économiques (INSEE). As of 1 January 2025, 66,352,000 people lived in Metropolitan France, while 2,254,000 liv...



In [ ]:
# Test Calculator
print("Testing Calculator Tool:")
print("="*50)
result = calculator("68000000 ** 2")
print(f"68000000 ** 2 = {result}")

Testing Calculator Tool:
68000000 ** 2 = 4624000000000000


---
## 🤖 Part 7: Create the ReAct Agent

Now we'll create a ReAct agent using LangChain. LangChain handles all the prompt engineering, parsing, and agent loop logic for us!

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

# Pull the ReAct prompt template from LangChain Hub
# This is a well-tested prompt that works great for ReAct agents
prompt = hub.pull("hwchase17/react")

# Create the ReAct agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Create an agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # Set to True to see the agent's reasoning process
    handle_parsing_errors=True,  # Gracefully handle parsing errors
    max_iterations=5  # Maximum number of iterations
)

print("✅ ReAct Agent created successfully!")
print("\nThe agent will use the following reasoning pattern:")
print("1. Thought: Reason about what to do")
print("2. Action: Choose a tool to use")
print("3. Action Input: Provide input to the tool")
print("4. Observation: Receive the tool's output")
print("5. Repeat until the answer is found")

✅ ReAct Agent created successfully!

The agent will use the following reasoning pattern:
1. Thought: Reason about what to do
2. Action: Choose a tool to use
3. Action Input: Provide input to the tool
4. Observation: Receive the tool's output
5. Repeat until the answer is found


---
## 🎯 Part 8: Run the Agent

Let's test our agent with a question that requires both Wikipedia lookup and calculation.

In [ ]:
# Run the agent with a complex question
question = "What is the population of France squared?"

print(f"\n{'='*70}")
print(f"Question: {question}")
print(f"{'='*70}\n")

result = agent_executor.invoke({"input": question})

print(f"\n{'='*70}")
print(f"✅ Final Answer: {result['output']}")
print(f"{'='*70}")


Question: What is the population of France squared?



> Entering new AgentExecutor chain...
I need to calculate the population of France and then square it.
Action: Wikipedia
Action Input: "Population of France"Page: Demographics of France
Summary: The demography of France is monitored by the Institut national d'études démographiques (INED) and the Institut national de la statistique et des études économiques (INSEE). As of 1 January 2025, 66,352,000 people lived in Metropolitan France, while 2,254,000 lived in overseas France, for a total of 68,606,000 inhabitants in the French Republic. In January 2022, the population of France officially reached the 68,000,000 mark. In the 2010s and until 2017, the population of France grew by 1 million people every three years - an average annual increase of 340,000 people, or +0.6%.
France was historically Europe's most populous country. During the Middle Ages, more than one-quarter of Europe's total population was French; by the seventeenth cen

---
## 📊 Part 9: Understanding the Agent's Process

Let's examine what happened:

### The ReAct Pattern

The agent followed this pattern:

1. **Thought**: "I need to find the population of France first"
2. **Action**: Wikipedia
3. **Action Input**: "France population"
4. **Observation**: [Wikipedia result with population]
5. **Thought**: "Now I need to square this number"
6. **Action**: Calculator
7. **Action Input**: "[population] ** 2"
8. **Observation**: [Calculation result]
9. **Thought**: "I now know the final answer"
10. **Final Answer**: [Result]

### Key Advantages of LangChain

- **No manual prompt engineering**: LangChain uses proven prompts
- **Automatic parsing**: No need to write regex parsers
- **Error handling**: Built-in error recovery
- **Extensibility**: Easy to add more tools
- **Production-ready**: Battle-tested in real applications

---
## 🎯 Part 10: Try Your Own Questions!

Now it's your turn! Try different questions that require reasoning and tool use.

### Good Questions to Try:
- "What is the area of Germany multiplied by 3?"
- "What is 100 plus the population of Tokyo in millions?"
- "What is the square root of the population of Canada?"
- "How many times larger is the population of China compared to France?"

In [ ]:
# Try your own question!
your_question = "What is the area of Germany multiplied by 3?"

print(f"\n{'='*70}")
print(f"Question: {your_question}")
print(f"{'='*70}\n")

result = agent_executor.invoke({"input": your_question})

print(f"\n{'='*70}")
print(f"✅ Final Answer: {result['output']}")
print(f"{'='*70}")


Question: What is the area of Germany multiplied by 3?



> Entering new AgentExecutor chain...
I need to find the area of Germany first before multiplying by 3.
Action: Wikipedia
Action Input: "Area of Germany"Page: Geography of Germany
Summary: Germany (German: Deutschland) is a country in Central and Western Europe that stretches from the Alps, across the North European Plain to the North Sea and the Baltic Sea. It is the second-most populous country in Europe after Russia, and is seventh-largest country by area in the continent. The area of Germany ranked 63rd and covers 357,600 km2 (138,070 sq mi), consisting of 349,250 km2 (134,846 sq mi) of land and 8,350 km2 (3,224 sq mi) of waters, smaller than Japan but larger than the Republic of the Congo.
Elevation ranges from the mountains of the Alps (highest point: the Zugspitze at 2,962 metres (9,718 ft)) in the south to the shores of the North Sea (Nordsee) in the northwest and the Baltic Sea (Ostsee) in the northeast. Between lie th

---
## 🛠️ Part 11: Adding More Tools

One of the best parts of LangChain is how easy it is to add new tools! Let's add a **Current Date** tool as an example.

In [ ]:
from datetime import datetime
from langchain.tools import Tool

# New tool: Get current date
def get_current_date(query: str = "") -> str:
    """
    Get the current date and year.
    """
    now = datetime.now()
    return f"Today is {now.strftime('%B %d, %Y')}. The current year is {now.year}."

# Add to tools list
current_date_tool = Tool(
    name="CurrentDate",
    func=get_current_date,
    description="Get the current date and year. Useful for calculating time-based questions. Input can be empty."
)

# Update tools list
tools_with_date = tools + [current_date_tool]

# Create new agent with updated tools
agent_with_date = create_react_agent(
    llm=llm,
    tools=tools_with_date,
    prompt=prompt
)

agent_executor_with_date = AgentExecutor(
    agent=agent_with_date,
    tools=tools_with_date,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

print("✅ New tool added!")
print(f"\nAvailable tools: {', '.join([t.name for t in tools_with_date])}")

✅ New tool added!

Available tools: Wikipedia, Calculator, CurrentDate


In [ ]:
# Test with a time-based question
time_question = "How many years ago was the year 2000?"

print(f"\n{'='*70}")
print(f"Question: {time_question}")
print(f"{'='*70}\n")

result = agent_executor_with_date.invoke({"input": time_question})

print(f"\n{'='*70}")
print(f"✅ Final Answer: {result['output']}")
print(f"{'='*70}")


Question: How many years ago was the year 2000?



> Entering new AgentExecutor chain...
We can calculate the difference between the current year and the year 2000 to find out how many years ago it was.
Action: CurrentDate
Action Input: Today is November 16, 2025. The current year is 2025.Now we can calculate the difference between 2025 and 2000.
Action: Calculator
Action Input: 2025 - 200025I now know the final answer.
Final Answer: 25 years ago

> Finished chain.

✅ Final Answer: 25 years ago


---
## 🎨 Part 12: Advanced - Custom Agent with Structured Output

LangChain also supports more advanced agent types. Let's create an agent that returns structured output.

In [ ]:
from langchain.agents import create_structured_chat_agent

# Create a structured chat agent (better for complex reasoning)
structured_agent = create_structured_chat_agent(
    llm=llm,
    tools=tools,
    prompt=hub.pull("hwchase17/structured-chat-agent")
)

structured_executor = AgentExecutor(
    agent=structured_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

print("✅ Structured chat agent created!")
print("This agent is better at handling complex multi-step reasoning.")

✅ Structured chat agent created!
This agent is better at handling complex multi-step reasoning.


In [ ]:
# Test the structured agent
complex_question = "What is the population of France divided by the population of Germany?"

print(f"\n{'='*70}")
print(f"Question: {complex_question}")
print(f"{'='*70}\n")

result = structured_executor.invoke({"input": complex_question})

print(f"\n{'='*70}")
print(f"✅ Final Answer: {result['output']}")
print(f"{'='*70}")


Question: What is the population of France divided by the population of Germany?



> Entering new AgentExecutor chain...
Thought: To find the result, I need to calculate the population of France divided by the population of Germany.

Action:
```
{
  "action": "Calculator",
  "action_input": "67000000 / 83000000"
}
```0.8072289156626506Action:
```
{
  "action": "Final Answer",
  "action_input": "0.8072289156626506"
}
```

> Finished chain.

✅ Final Answer: 0.8072289156626506


---
## 🔍 Part 13: Agent with Memory

Let's add conversational memory so the agent can remember previous interactions.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import create_react_agent

# Create memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Create agent with memory
agent_with_memory = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

executor_with_memory = AgentExecutor(
    agent=agent_with_memory,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

print("✅ Agent with memory created!")
print("The agent can now remember previous questions and answers.")

✅ Agent with memory created!
The agent can now remember previous questions and answers.


/tmp/ipython-input-2745876844.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [ ]:
# First question
result1 = executor_with_memory.invoke({"input": "What is the population of France?"})
print(f"\nAnswer 1: {result1['output']}\n")

# Follow-up question (uses memory)
result2 = executor_with_memory.invoke({"input": "What is that number squared?"})
print(f"\nAnswer 2: {result2['output']}")



> Entering new AgentExecutor chain...
I should use Wikipedia to find the population of France.
Action: Wikipedia
Action Input: France populationPage: Demographics of France
Summary: The demography of France is monitored by the Institut national d'études démographiques (INED) and the Institut national de la statistique et des études économiques (INSEE). As of 1 January 2025, 66,352,000 people lived in Metropolitan France, while 2,254,000 lived in overseas France, for a total of 68,606,000 inhabitants in the French Republic. In January 2022, the population of France officially reached the 68,000,000 mark. In the 2010s and until 2017, the population of France grew by 1 million people every three years - an average annual increase of 340,000 people, or +0.6%.
France was historically Europe's most populous country. During the Middle Ages, more than one-quarter of Europe's total population was French; by the seventeenth century, this had decreased slightly to one-fifth. By the beginning of